# Assignment-16-Neural networks-forest fire

In [1]:
!pip install keras==2.3.1

Defaulting to user installation because normal site-packages is not writeable
  Using cached Keras-2.3.1-py2.py3-none-any.whl (377 kB)
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 2.3.1 which is incompatible.


In [2]:
!pip install tensorflow


Defaulting to user installation because normal site-packages is not writeable
  Using cached keras-2.15.0-py3-none-any.whl (1.7 MB)
  Attempting uninstall: keras
    Found existing installation: Keras 2.3.1
    Uninstalling Keras-2.3.1:
      Successfully uninstalled Keras-2.3.1


In [3]:
#Import Libraries
import numpy as np
import pandas as pd 
import os
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
from tqdm import tqdm


from keras import Sequential
from keras import optimizers
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential,Model
from keras import backend as K

from keras import initializers, regularizers, constraints
from sklearn.model_selection import KFold, cross_val_score, train_test_split

In [4]:
#Import Dataset
data = pd.read_csv('forestfires.csv')
data

,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,...,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
0,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,...,0,0,0,0,1,0,0,0,0,small
1,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,...,0,0,0,0,0,0,0,1,0,small
2,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,...,0,0,0,0,0,0,0,1,0,small
3,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,...,0,0,0,0,1,0,0,0,0,small
4,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,...,0,0,0,0,1,0,0,0,0,small
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,aug,sun,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,...,0,0,0,0,0,0,0,0,0,large
513,aug,sun,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,...,0,0,0,0,0,0,0,0,0,large
514,aug,sun,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,...,0,0,0,0,0,0,0,0,0,large
515,aug,sat,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,...,0,0,0,0,0,0,0,0,0,small


In [5]:
data = data.drop(['month','day'],axis=1)

In [6]:
data.head()

,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,dayfri,...,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
0,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0,1,...,0,0,0,0,1,0,0,0,0,small
1,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0,0,...,0,0,0,0,0,0,0,1,0,small
2,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0,0,...,0,0,0,0,0,0,0,1,0,small
3,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0,1,...,0,0,0,0,1,0,0,0,0,small
4,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0,0,...,0,0,0,0,1,0,0,0,0,small


In [7]:
mapping = {'small':1,'large':2}

In [8]:
data = data.replace(mapping)

In [9]:
data.head()

,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,dayfri,...,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
0,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0,1,...,0,0,0,0,1,0,0,0,0,1
1,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0,0,...,0,0,0,0,0,0,0,1,0,1
2,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0,0,...,0,0,0,0,0,0,0,1,0,1
3,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0,1,...,0,0,0,0,1,0,0,0,0,1
4,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0,0,...,0,0,0,0,1,0,0,0,0,1


In [10]:
x = np.array(data.iloc[:,0:28])
y = np.array(data.iloc[:,28])

In [11]:
def norm_func(i):
    x = (i-i.min())/(i.max()-i.min())
    return (x)

In [12]:
x_norm = norm_func(x)

In [13]:
xtrain,xtest,ytrain,ytest = train_test_split(x_norm,y,test_size=0.3, stratify= y)

In [15]:
from keras.layers import Activation, Dense

In [16]:
model = Sequential()
model.add(Dense(8, input_dim=28 , activation= 'linear'))
model.add(Dense(4, activation= 'tanh'))
model.add(Dense(1, activation= 'sigmoid'))

In [17]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [18]:
history = model.fit(xtrain,ytrain, validation_split= 0.3,epochs=120, batch_size=10)

Epoch 1/120


26/26 [==============================] - 3s 27ms/step - loss: 0.5437 - accuracy: 0.7103 - val_loss: 0.4799 - val_accuracy: 0.7706
Epoch 2/120
26/26 [==============================] - 0s 7ms/step - loss: 0.3927 - accuracy: 0.7143 - val_loss: 0.3414 - val_accuracy: 0.7706
Epoch 3/120
26/26 [==============================] - 0s 7ms/step - loss: 0.2400 - accuracy: 0.7143 - val_loss: 0.2048 - val_accuracy: 0.7706
Epoch 4/120
26/26 [==============================] - 0s 7ms/step - loss: 0.0866 - accuracy: 0.7143 - val_loss: 0.0668 - val_accuracy: 0.7706
Epoch 5/120
26/26 [==============================] - 0s 7ms/step - loss: -0.0645 - accuracy: 0.7143 - val_loss: -0.0608 - val_accuracy: 0.7706
Epoch 6/120
26/26 [==============================] - 0s 7ms/step - loss: -0.1992 - accuracy: 0.7143 - val_loss: -0.1813 - val_accuracy: 0.7706
Epoch 7/120
26/26 [==============================] - 0s 7ms/step - loss: -0.3311 - accuracy: 0.7143 - val_loss: -0.2866 - val_accuracy: 0.7706
Epoc

26/26 [==============================] - 0s 9ms/step - loss: -2.3400 - accuracy: 0.7143 - val_loss: -1.8918 - val_accuracy: 0.7706
Epoch 55/120
26/26 [==============================] - 0s 9ms/step - loss: -2.3724 - accuracy: 0.7143 - val_loss: -1.9197 - val_accuracy: 0.7706
Epoch 56/120
26/26 [==============================] - 0s 9ms/step - loss: -2.4051 - accuracy: 0.7143 - val_loss: -1.9459 - val_accuracy: 0.7706
Epoch 57/120
26/26 [==============================] - 0s 9ms/step - loss: -2.4377 - accuracy: 0.7143 - val_loss: -1.9710 - val_accuracy: 0.7706
Epoch 58/120
26/26 [==============================] - 0s 8ms/step - loss: -2.4711 - accuracy: 0.7143 - val_loss: -1.9968 - val_accuracy: 0.7706
Epoch 59/120
26/26 [==============================] - 0s 7ms/step - loss: -2.5030 - accuracy: 0.7143 - val_loss: -2.0225 - val_accuracy: 0.7706
Epoch 60/120
26/26 [==============================] - 0s 6ms/step - loss: -2.5345 - accuracy: 0.7143 - val_loss: -2.0494 - val_accuracy: 0.7706
Epoch

Epoch 111/120
26/26 [==============================] - 0s 6ms/step - loss: -4.2119 - accuracy: 0.7143 - val_loss: -3.3942 - val_accuracy: 0.7706
Epoch 112/120
26/26 [==============================] - 0s 6ms/step - loss: -4.2432 - accuracy: 0.7143 - val_loss: -3.4199 - val_accuracy: 0.7706
Epoch 113/120
26/26 [==============================] - 0s 6ms/step - loss: -4.2763 - accuracy: 0.7143 - val_loss: -3.4490 - val_accuracy: 0.7706
Epoch 114/120
26/26 [==============================] - 0s 6ms/step - loss: -4.3110 - accuracy: 0.7143 - val_loss: -3.4756 - val_accuracy: 0.7706
Epoch 115/120
26/26 [==============================] - 0s 7ms/step - loss: -4.3453 - accuracy: 0.7143 - val_loss: -3.5032 - val_accuracy: 0.7706
Epoch 116/120
26/26 [==============================] - 0s 7ms/step - loss: -4.3797 - accuracy: 0.7143 - val_loss: -3.5283 - val_accuracy: 0.7706
Epoch 117/120
26/26 [==============================] - 0s 7ms/step - loss: -4.4117 - accuracy: 0.7143 - val_loss: -3.5559 - val_ac

In [20]:
scores = model.evaluate(xtrain, ytrain)
print("%s: %.2f%%" %(model.metrics_names[1],scores[1]*100))

12/12 [==============================] - 0s 4ms/step - loss: -4.2593 - accuracy: 0.7313
accuracy: 73.13%


In [21]:
scores = model.evaluate(xtest,ytest)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

5/5 [==============================] - 0s 6ms/step - loss: -4.2676 - accuracy: 0.7308
accuracy: 73.08%
